# Problems:
1. Find the Province & City where the "Data science" related job postings are high.
2. Find the companies who have Data science related positions.
3. Find the Rental Apartments near to the companies.
4. find the best neighbourhoods for each company. Neighbourhoods should have following venues in 3km radius.
    a. Gym
    b. Coffee shop
    c. Bar and Restaurant 
    d. Grocery store etc.

In [70]:

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
Jobs = pd.read_csv('data_acquisition/jobs/indeed.csv')
Jobs.head()

,Title,Location,Company
0,"Data Scientist, Video Analytics","Oakville, ON",Geotab
1,Tech- Data Scientist & Client Integration Asso...,"Toronto, ON",Exiger
2,Data Scientist/Bioinformatician,"Toronto, ON",SickKids
3,data scientist,"Vancouver, BC","ResponsiveAds, Inc."
4,Sr Data Scientist,"Kirkland, QC",IQVIA


## cleaning Data 

In [3]:
# Separate City and Province from Location
Jobs1 = pd.concat([Jobs['Title'],Jobs['Company'], Jobs['Location'].str.split(', ', expand=True), ], axis=1)

In [4]:
Jobs1.head()

,Title,Company,0,1
0,"Data Scientist, Video Analytics",Geotab,Oakville,ON
1,Tech- Data Scientist & Client Integration Asso...,Exiger,Toronto,ON
2,Data Scientist/Bioinformatician,SickKids,Toronto,ON
3,data scientist,"ResponsiveAds, Inc.",Vancouver,BC
4,Sr Data Scientist,IQVIA,Kirkland,QC


In [5]:
Jobs1.columns = ['Job Title', 'Company Name', 'City', 'Province Code']
Jobs1.to_csv("data_acquisition/jobs/cleanIndeed.csv",index=False)

# Tableau Results: 
## Tableau is a visual analytics platform which helps to understand the data faster and better with many chart options. 
<img src="bubble.png" width=400 height=600 align="center"/>
<img src="textTable.png" width=400 height=600 align="center"/>
<img src="sidebyside.png" width=1000 height=700 align="center"/>


 ## Findings
Based on the above Charts, I found that Ontario Province has large number of Data science related openings and Toronto is the capital city of Ontario Province, it has much more jobs. I am very happy with the findings of the above. Ontario province is the best option for me to immigrate. 

In [6]:
Jobs1.head()

,Job Title,Company Name,City,Province Code
0,"Data Scientist, Video Analytics",Geotab,Oakville,ON
1,Tech- Data Scientist & Client Integration Asso...,Exiger,Toronto,ON
2,Data Scientist/Bioinformatician,SickKids,Toronto,ON
3,data scientist,"ResponsiveAds, Inc.",Vancouver,BC
4,Sr Data Scientist,IQVIA,Kirkland,QC


In [7]:
Jobs1.shape

(1870, 4)

### Droping unwanted columns

I have already found the city based on large number of job openings. Toronto is the suitable city for me to move. I would like to drop Job Title column and only keeping toronto city.

In [8]:
Jobs2 = Jobs1.drop(['Job Title'], axis=1)
Jobs2.head()

,Company Name,City,Province Code
0,Geotab,Oakville,ON
1,Exiger,Toronto,ON
2,SickKids,Toronto,ON
3,"ResponsiveAds, Inc.",Vancouver,BC
4,IQVIA,Kirkland,QC


In [9]:
# removing nan 
Jobs2.dropna(inplace= True)
Jobs2.shape # it was 1870 rows and 1848 after removing nan

(1848, 3)

In [10]:
#removing all cities except Toronto
Jobs2 = Jobs2.loc[Jobs2['City'] == 'Toronto']
Jobs2.head()

,Company Name,City,Province Code
1,Exiger,Toronto,ON
2,SickKids,Toronto,ON
11,Coursera,Toronto,ON
15,TKEES,Toronto,ON
20,SickKids,Toronto,ON


In [11]:
Jobs2.shape

(627, 3)

In [12]:
# assending order of company names
Jobs2.sort_values("Company Name", axis= 0, ascending = True, inplace = True, kind= 'quicksort', na_position= 'last')
Jobs2.head()

,Company Name,City,Province Code
280,"AMZN CAN Fulfillment Svcs, ULC",Toronto,ON
169,"AMZN CAN Fulfillment Svcs, ULC",Toronto,ON
185,"AMZN CAN Fulfillment Svcs, ULC",Toronto,ON
402,"AMZN CAN Fulfillment Svcs, ULC",Toronto,ON
26,"AMZN CAN Fulfillment Svcs, ULC",Toronto,ON


In [13]:
#we can see that Jobs2 dataframe has multiple Job openings within the same company
#let's remove duplicates
Jobs3 = Jobs2.drop_duplicates()
Jobs3

,Company Name,City,Province Code
280,"AMZN CAN Fulfillment Svcs, ULC",Toronto,ON
57,Accenture,Toronto,ON
474,BFS Capital,Toronto,ON
307,BMO Financial Group,Toronto,ON
525,BMW Group Financial Services Canada,Toronto,ON
983,Birdseye,Toronto,ON
487,Capgemini,Toronto,ON
1055,Ceridian,Toronto,ON
117,Colliers International,Toronto,ON
63,Coursera,Toronto,ON


In [14]:
Jobs3.shape

(28, 3)

In [15]:
# reset index
Jobs3.reset_index(drop = True)
Jobs3.to_excel("company1.xlsx",index=False)

# get latitude and longitude of each company using geocode

In [16]:
#merge columns using Concat
df1 = pd.DataFrame(columns = ['Address'])
df1['Address'] = pd.concat([Jobs3["Company Name"] + " , " + Jobs3["City"] + " , "+ Jobs3["Province Code"]], ignore_index= True, sort= False)
df1.to_excel("company.xlsx",index=False)
df1

,Address
0,"AMZN CAN Fulfillment Svcs, ULC , Toronto , ON"
1,"Accenture , Toronto , ON"
2,"BFS Capital , Toronto , ON"
3,"BMO Financial Group , Toronto , ON"
4,"BMW Group Financial Services Canada , Toronto ..."
5,"Birdseye , Toronto , ON"
6,"Capgemini , Toronto , ON"
7,"Ceridian , Toronto , ON"
8,"Colliers International , Toronto , ON"
9,"Coursera , Toronto , ON"


In [17]:
!pip install googlemaps

from googlemaps import Client as GoogleMaps
import pandas as pd 

In [18]:
gmaps = GoogleMaps('key')
addresses = pd.read_excel("company.xlsx")
addresses['lat'] = ""
addresses['long'] = ""
for x in range(len(addresses)):
    try:
        geocode_result = gmaps.geocode(addresses['Address'][x])
        addresses['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
        addresses['long'][x] = geocode_result[0]['geometry']['location']['lng']
    except IndexError:
        print("Address was wrong...")
    except Exception as e:
        print("Unexpected error occurred.", e )
addresses

,Address,lat,long
0,"AMZN CAN Fulfillment Svcs, ULC , Toronto , ON",43.6532,-79.3832
1,"Accenture , Toronto , ON",43.6496,-79.3796
2,"BFS Capital , Toronto , ON",43.6481,-79.3847
3,"BMO Financial Group , Toronto , ON",43.6486,-79.3817
4,"BMW Group Financial Services Canada , Toronto ...",43.8875,-79.3823
5,"Birdseye , Toronto , ON",43.6766,-79.5878
6,"Capgemini , Toronto , ON",43.6497,-79.3867
7,"Ceridian , Toronto , ON",43.7463,-79.409
8,"Colliers International , Toronto , ON",43.6472,-79.3791
9,"Coursera , Toronto , ON",43.6518,-79.3802


In [21]:
addresses.to_excel("data_acquisition/jobs/company.xlsx")
#exported geocodes to company.excel 

#addresses = pd.read_excel("data_acquisition/jobs/company.xlsx")
#addresses

,Unnamed: 0,Address,lat,long
0,0,"AMZN CAN Fulfillment Svcs, ULC , Toronto , ON",43.653226,-79.383184
1,1,"Accenture , Toronto , ON",43.649600,-79.379575
2,2,"BFS Capital , Toronto , ON",43.648133,-79.384655
3,3,"BMO Financial Group , Toronto , ON",43.648636,-79.381744
4,4,"BMW Group Financial Services Canada , Toronto ...",43.887468,-79.382294
5,5,"Birdseye , Toronto , ON",43.676579,-79.587798
6,6,"Capgemini , Toronto , ON",43.649707,-79.386675
7,7,"Ceridian , Toronto , ON",43.746331,-79.408965
8,8,"Colliers International , Toronto , ON",43.647155,-79.379150
9,9,"Coursera , Toronto , ON",43.651791,-79.380169


In [22]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm

import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans


!pip install geopandas
!pip install geopy

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [23]:
#toronto geocodes
latitude = 43.6532
longitude= -79.3832

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lt, lg, label in zip(addresses['lat'], addresses['long'], addresses['Address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lt, lg],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

My residential preferences are :
Rent between 1000 to 2000, Pets = yes, Bedroom = 1, Type = condos or apartment.
scraped data from Rental.ca  #https://rentals.ca/toronto?rentrange=1000-2000&types=condo&types=apartment&beds=1%2B&pets=true
rental.xlsx have all the data about available residential places


In [25]:
rental_data = pd.read_excel('data_acquisition/rental_data/rental_data.xlsx')
rental_data.head()

,Unnamed: 0,Rent,Location,lat,long
0,0,"$1,750","70 Temperance St, Unit 4116,Toronto,Ontario",43.650530,-79.382148
1,1,"$1,900","197 Yonge Street St, Unit 2104,Toronto,Ontario",43.653525,-79.379238
2,2,"$1,700","403 Church St, Unit 1711,Toronto,Ontario",43.662066,-79.379337
3,3,"$2,100","25 Oxley St, Unit 413,Toronto,Ontario",43.646247,-79.394528
4,4,"$1,950","87 Peter St, Unit 2701,Toronto,Ontario",43.646944,-79.392397


In [27]:
rental_data['latlong'] = rental_data[rental_data.columns[3:]].apply(lambda x: ','.join(x.dropna().astype(float).astype(str)), axis=1)
rental_data.head()

,Unnamed: 0,Rent,Location,lat,long,latlong
0,0,"$1,750","70 Temperance St, Unit 4116,Toronto,Ontario",43.650530,-79.382148,"43.6505301,-79.3821482"
1,1,"$1,900","197 Yonge Street St, Unit 2104,Toronto,Ontario",43.653525,-79.379238,"43.6535246,-79.3792385"
2,2,"$1,700","403 Church St, Unit 1711,Toronto,Ontario",43.662066,-79.379337,"43.6620656,-79.37933710000001"
3,3,"$2,100","25 Oxley St, Unit 413,Toronto,Ontario",43.646247,-79.394528,"43.6462468,-79.39452849999999"
4,4,"$1,950","87 Peter St, Unit 2701,Toronto,Ontario",43.646944,-79.392397,"43.6469441,-79.3923973"


In [ ]:

from geopy.geocoders import Nominatim


rental_data['Address'] = ""
latlong1 = rental_data['latlong']

for i in range(0,660):
    try:
        geolocator = Nominatim(user_agent="cap")
        rental_data['Address'][i] = geolocator.geocode(rental_data['latlong'][i])
    except IndexError:
        print("Address was wrong...")

rental_data

In [54]:
rental_data.to_excel('data_acquisition/rental_data/RentReverse.xlsx')

In [55]:
neighbourhoods = pd.read_excel('data_acquisition/neighbourhoods/Neighbourhoods.xlsx')
rental_houses = pd.read_excel('data_acquisition/rental_data/RentReverse1.xlsx')
neighbourhoods.head(1)


,Unnamed: 0,PostalCode,Borough,Neighbourhood
0,1,M4N,Central Toronto,Lawrence Park


In [56]:
rental_houses.rename(columns={'postalcode': 'postalcode'})

,Unnamed: 0,Unnamed: 0.1,Rent,Location,lat,long,latlong,Address,postalcode,Neighbourhood
0,0,0,"1,750","70 Temperance St, Unit 4116,Toronto,Ontario",43.650530,-79.382148,"43.650530100000005,-79.3821482","INDX, 70, Temperance Street, Toronto Centre, O...",M5H,NaN
1,1,1,"1,900","197 Yonge Street St, Unit 2104,Toronto,Ontario",43.653525,-79.379238,"43.653524600000004,-79.3792385","Bank of Toronto (former), 205, Yonge Street, T...",M5H,NaN
2,2,2,"1,700","403 Church St, Unit 1711,Toronto,Ontario",43.662066,-79.379337,"43.6620656,-79.37933710000001","438, Church Street, Toronto Centre, Old Toront...",M5B,NaN
3,3,3,"2,100","25 Oxley St, Unit 413,Toronto,Ontario",43.646247,-79.394528,"43.6462468,-79.39452849999998","Glas Condominium, Oxley Street, Spadina—Fort Y...",M5V,NaN
4,4,4,"1,950","87 Peter St, Unit 2701,Toronto,Ontario",43.646944,-79.392397,"43.6469441,-79.3923973","The Commodore, Adelaide Street West, Spadina—F...",M5H,NaN
5,5,5,"1,688","215 Queen St W St, Unit 1515,Toronto,Ontario",43.650484,-79.387860,"43.6504836,-79.3878601","215, Queen Street West, Spadina—Fort York, Old...",M5H,NaN
6,6,6,"1,800","18 Yonge St, Unit 3609,Toronto,Ontario",43.644408,-79.376562,"43.644408399999996,-79.37656170000001","18, Yonge Street, Spadina—Fort York, Old Toron...",M5E,NaN
7,7,7,"1,950","318 Richmond St W, Unit 3103,Toronto,Ontario",43.648900,-79.392205,"43.6489001,-79.39220499999998","320, Richmond Street West, Spadina—Fort York, ...",M5V,NaN
8,8,8,"1,850","295 Adelaide St W, Unit 2711,Toronto,Ontario",43.647627,-79.390408,"43.647627299999996,-79.39040809999999","FiGO, Adelaide Street West, Spadina—Fort York,...",M5H,NaN
9,9,9,"1,800","87 Peter St, Unit 1815,Toronto,Ontario",43.646944,-79.392397,"43.6469441,-79.3923973","The Commodore, Adelaide Street West, Spadina—F...",M5H,NaN


In [57]:
#matching neighbourhoods of rental houses and neighbourhoods dataframe based on postal codes

neigh = []
for i in range(0,659):
    for a in range(0,39):
        if rental_houses['postalcode'][i] == neighbourhoods['PostalCode'][a]:
            print(rental_houses['postalcode'][i], neighbourhoods['PostalCode'][a], neighbourhoods['Neighbourhood'][a])
            nei = neighbourhoods['Neighbourhood'][a]
            neigh.append((nei))
rental_houses1 = pd.DataFrame(neigh, columns= ['Neighbourhood'])

M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M5B M5B Garden District, Ryerson
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M5E M5E Berczy Park
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M5B M5B Garden District, Ryerson
M5B M5B Garden District, Ryerson
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5T M5T Kensington Market, Chinatown, Grange Park
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5B M5B Garden District, Ryerson
M5E M5E Berczy Park
M5J M5J Harbourfront East, Union Station, Toronto Islands
M5B M5B Garden District, Ryer

M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5B M5B Garden District, Ryerson
M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M4Y M4Y Church and Wellesley
M5B M5B Garden District, Ryerson
M5T M5T Kensington Market, Chinatown, Grange Park
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5B M5B Garden District, Ryerson
M5H M5H Richmond, Adelaide, King
M5B M5B Garden District, Ryerson
M5B M5B Garden District, Ryerson
M5H M5H Richmond, Adelaide, King
M5G M5G Central Bay Street
M5G M5G Central Bay Street
M5B M5B Garden District, Ryerson
M5E M5E Berczy Park
M5H M5H Richmond, Adelaide, King
M5B M5B Garden District, Ryerson
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island

M5B M5B Garden District, Ryerson
M5C M5C St. James Town
M5B M5B Garden District, Ryerson
M5H M5H Richmond, Adelaide, King
M5B M5B Garden District, Ryerson
M5E M5E Berczy Park
M5E M5E Berczy Park
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5J M5J Harbourfront East, Union Station, Toronto Islands
M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M5E M5E Berczy Park
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5T M5T Kensington Market, Chinatown, Grange Park
M5H M5H Richmond, Adelaide, King
M5H M5H Richmond, Adelaide, King
M5B M5B Garden District, Ryerson
M5G M5G Central Bay Street
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5V M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5H M5H Richmond, Adelaid

In [58]:
rental_houses.head(1)

,Unnamed: 0,Unnamed: 0.1,Rent,Location,lat,long,latlong,Address,postalcode,Neighbourhood
0,0,0,"1,750","70 Temperance St, Unit 4116,Toronto,Ontario",43.65053,-79.382148,"43.650530100000005,-79.3821482","INDX, 70, Temperance Street, Toronto Centre, O...",M5H,NaN


In [59]:
rental_houses.drop(['Neighbourhood','Unnamed: 0','Unnamed: 0.1' ], axis=1,inplace= True)
rental_houses.head(1)

,Rent,Location,lat,long,latlong,Address,postalcode
0,"1,750","70 Temperance St, Unit 4116,Toronto,Ontario",43.65053,-79.382148,"43.650530100000005,-79.3821482","INDX, 70, Temperance Street, Toronto Centre, O...",M5H


In [60]:
rental_houses= pd.concat([rental_houses, rental_houses1], axis=1)
rental_houses

,Rent,Location,lat,long,latlong,Address,postalcode,Neighbourhood
0,"1,750","70 Temperance St, Unit 4116,Toronto,Ontario",43.650530,-79.382148,"43.650530100000005,-79.3821482","INDX, 70, Temperance Street, Toronto Centre, O...",M5H,"Richmond, Adelaide, King"
1,"1,900","197 Yonge Street St, Unit 2104,Toronto,Ontario",43.653525,-79.379238,"43.653524600000004,-79.3792385","Bank of Toronto (former), 205, Yonge Street, T...",M5H,"Richmond, Adelaide, King"
2,"1,700","403 Church St, Unit 1711,Toronto,Ontario",43.662066,-79.379337,"43.6620656,-79.37933710000001","438, Church Street, Toronto Centre, Old Toront...",M5B,"Garden District, Ryerson"
3,"2,100","25 Oxley St, Unit 413,Toronto,Ontario",43.646247,-79.394528,"43.6462468,-79.39452849999998","Glas Condominium, Oxley Street, Spadina—Fort Y...",M5V,"CN Tower, King and Spadina, Railway Lands, Har..."
4,"1,950","87 Peter St, Unit 2701,Toronto,Ontario",43.646944,-79.392397,"43.6469441,-79.3923973","The Commodore, Adelaide Street West, Spadina—F...",M5H,"Richmond, Adelaide, King"
5,"1,688","215 Queen St W St, Unit 1515,Toronto,Ontario",43.650484,-79.387860,"43.6504836,-79.3878601","215, Queen Street West, Spadina—Fort York, Old...",M5H,"Richmond, Adelaide, King"
6,"1,800","18 Yonge St, Unit 3609,Toronto,Ontario",43.644408,-79.376562,"43.644408399999996,-79.37656170000001","18, Yonge Street, Spadina—Fort York, Old Toron...",M5E,Berczy Park
7,"1,950","318 Richmond St W, Unit 3103,Toronto,Ontario",43.648900,-79.392205,"43.6489001,-79.39220499999998","320, Richmond Street West, Spadina—Fort York, ...",M5V,"CN Tower, King and Spadina, Railway Lands, Har..."
8,"1,850","295 Adelaide St W, Unit 2711,Toronto,Ontario",43.647627,-79.390408,"43.647627299999996,-79.39040809999999","FiGO, Adelaide Street West, Spadina—Fort York,...",M5H,"Richmond, Adelaide, King"
9,"1,800","87 Peter St, Unit 1815,Toronto,Ontario",43.646944,-79.392397,"43.6469441,-79.3923973","The Commodore, Adelaide Street West, Spadina—F...",M5H,"Richmond, Adelaide, King"


In [61]:
rental_houses.drop(['lat', 'long'], axis=1, inplace= True)

In [62]:
rental_houses.shape

(660, 6)

In [ ]:
!pip install googlemaps
from googlemaps import Client as GoogleMaps

gmaps = GoogleMaps('key')
rental_houses['Nei_lat'] = ""
rental_houses['Nei_long'] = ""
for x in range(0, 659):
    try:
        geocode_result = gmaps.geocode(rental_houses['Neighbourhood'][x])
        #print(geocode_result)
      #  print(rental_houses['Neighbourhood'][x])
        rental_houses['Nei_lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
        rental_houses['Nei_long'][x] = geocode_result[0]['geometry']['location']['lng']
    except IndexError:
        print("Address was wrong...")



In [ ]:
rental_clean = rental_houses.drop(['Address','postalcode'], axis=1)
grouped = rental_clean.groupby('Neighbourhood')
grouped.describe()

In [ ]:
unique_n = rental_clean.drop_duplicates(subset=['Neighbourhood'])

In [65]:
unique_n = pd.read_excel('data_acquisition/neighbourhoods/unique_n.xlsx')
unique2 = unique_n.drop(['Location', 'Rent', 'latlong'], axis=1)
unique_n

,Unnamed: 0,Unnamed: 0.1,Rent,Location,latlong,Neighbourhood,Nei_lat,Nei_long
0,0,0,"1,750","70 Temperance St, Unit 4116,Toronto,Ontario","43.650530100000005,-79.3821482","Richmond, Adelaide, King",43.649972,-79.383223
1,2,2,"1,700","403 Church St, Unit 1711,Toronto,Ontario","43.6620656,-79.37933710000001","Garden District, Ryerson",43.657658,-79.378802
2,3,3,"2,100","25 Oxley St, Unit 413,Toronto,Ontario","43.6462468,-79.39452849999998","CN Tower, King and Spadina, Railway Lands, Har...",43.642566,-79.387057
3,6,6,"1,800","18 Yonge St, Unit 3609,Toronto,Ontario","43.644408399999996,-79.37656170000001",Berczy Park,43.648100,-79.375200
4,14,14,"1,800","426 University Ave, Unit 2306,Toronto,Ontario","43.6539487,-79.38857010000002","Kensington Market, Chinatown, Grange Park",43.650883,-79.397226
5,18,18,"1,800","55 Bremner Blvd, Unit 3609,Toronto,Ontario","43.6428849,-79.3806168","Harbourfront East, Union Station, Toronto Islands",43.640412,-79.375724
6,24,24,"1,900","832 Bay St, Unit 4201,Toronto,Ontario","43.661927,-79.3865112","University of Toronto, Harbord",43.662892,-79.395656
7,27,27,"2,100","38 Elm St, Unit 315,Toronto,Ontario","43.6575994,-79.38364",Central Bay Street,43.657298,-79.384364
8,28,28,"1,800","105 Victoria St, Unit Lph4,Toronto,Ontario","43.6498151,-79.3599261","Regent Park, Harbourfront",43.660323,-79.362044
9,44,44,"2,000","15 Grenville St, Unit 1508,Toronto,Ontario","43.6618822,-79.3841302",Church and Wellesley,43.664542,-79.381752


In [69]:
unique2 = unique2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
unique2.to_excel('data_acquisition/neighbourhoods/unique2.xlsx')

In [102]:
Toronto_Neighbourhoods = pd.read_excel('unique2.xlsx')
Toronto_Neighbourhoods


,Unnamed: 0,Neighbourhood,Nei_lat,Nei_long
0,0,"Richmond, Adelaide, King",43.649972,-79.383223
1,1,"Garden District, Ryerson",43.657658,-79.378802
2,2,"CN Tower, King and Spadina, Railway Lands, Har...",43.642566,-79.387057
3,3,Berczy Park,43.648100,-79.375200
4,4,"Kensington Market, Chinatown, Grange Park",43.650883,-79.397226
5,5,"Harbourfront East, Union Station, Toronto Islands",43.640412,-79.375724
6,6,"University of Toronto, Harbord",43.662892,-79.395656
7,7,Central Bay Street,43.657298,-79.384364
8,8,"Regent Park, Harbourfront",43.660323,-79.362044
9,9,Church and Wellesley,43.664542,-79.381752


In [103]:
#Toronto coordinates 
latitude = 43.6532
longitude= -79.3832

map_toronto = folium.Map(location = [latitude, longitude], zoom_start=1)

#add markers to the map

for lat, lng, label in zip(Toronto_Neighbourhoods['Nei_lat'], Toronto_Neighbourhoods['Nei_long'], Toronto_Neighbourhoods['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [104]:
CLIENT_ID = 'your id' # your Foursquare ID
CLIENT_SECRET = 'secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your id
CLIENT_SECRET:secret


In [105]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [106]:
toronto_venues = getNearbyVenues(names=Toronto_Neighbourhoods['Neighbourhood'],
                                   latitudes=Toronto_Neighbourhoods['Nei_lat'],
                                   longitudes=Toronto_Neighbourhoods['Nei_long']
                                  )

Richmond, Adelaide, King


KeyError: 'groups'

In [ ]:
print(toronto_venues.shape)
toronto_venues
toronto_venues.to_excel('data_acquisition/neighbourhoods/toronto_venues1.xlsx')

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding categorical to the form(0 and 1 ) for ML algorthims
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

In [107]:
toronto_onehot.shape

(814, 186)

In [108]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Historic Site,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.010000,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.000000,0.01,0.01,0.030000,0.000000,0.000000,0.000000,0.00000,0.030000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.00,0.030000,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.020000,0.00000,0.01,0.000000,0.020000,0.090000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.000000,0.020000,0.000000,0.010000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.00000,0.010000,0.01,0.010000,0.00000,0.000000,0.000000,0.00,0.030000,0.000000,0.00,0.000000,0.00,0.01,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.040000,0.000000,0.000000,0.000000,0.000000,0.050000,0.040000,0.01,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.010000,0.01,0.000000,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.01,0.01000,0.000000,0.000000,0.0

In [109]:
toronto_grouped.shape

(12, 186)

In [110]:
num_top_venues = 100

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp.to_excel('freq.xlsx')
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
temp.to_excel('freq1.xlsx')

----Berczy Park----
                              venue  freq
0                       Coffee Shop  0.09
1                        Restaurant  0.06
2                              Café  0.05
3                Italian Restaurant  0.05
4               Japanese Restaurant  0.04
5                             Hotel  0.04
6                          Beer Bar  0.03
7                         Gastropub  0.03
8                Seafood Restaurant  0.03
9                               Gym  0.03
10                           Bakery  0.03
11                   Breakfast Spot  0.03
12                   Farmers Market  0.02
13                     Cocktail Bar  0.02
14                      Cheese Shop  0.02
15                         Creperie  0.02
16                             Park  0.02
17                      Art Gallery  0.02
18                     Optical Shop  0.01
19              Sporting Goods Shop  0.01
20                    Deli / Bodega  0.01
21                 Department Store  0.01
22            

                              venue  freq
0                               Bar  0.06
1                       Yoga Studio  0.04
2                              Café  0.04
3               Arts & Crafts Store  0.03
4     Vegetarian / Vegan Restaurant  0.03
5                            Bistro  0.02
6                    Sandwich Place  0.02
7                       Coffee Shop  0.02
8               Dumpling Restaurant  0.02
9                    Clothing Store  0.02
10                 Sushi Restaurant  0.02
11                      Pizza Place  0.02
12                French Restaurant  0.02
13                         Boutique  0.02
14                       Board Shop  0.02
15                             Park  0.02
16                   Cosmetics Shop  0.02
17               Mexican Restaurant  0.02
18                      Art Gallery  0.02
19            Vietnamese Restaurant  0.02
20                 Ramen Restaurant  0.01
21                         Creperie  0.01
22                     Dance Studi

In [111]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 100

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue,51th Most Common Venue,52th Most Common Venue,53th Most Common Venue,54th Most Common Venue,55th Most Common Venue,56th Most Common Venue,57th Most Common Venue,58th Most Common Venue,59th Most Common Venue,60th Most Common Venue,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue,71th Most Common Venue,72th Most Common Venue,73th Most Common Venue,74th Most Common Venue,75th Most Common Venue,76th Most Common Venue,77th Most Common Venue,78th Most Common Venue,79th Most Common Venue,80th Most Common Venue,81th Most Common Venue,82th Most Common Venue,83th Most Common Venue,84th Most Common Venue,85th Most Common Venue,86th Most Common Venue,87th Most Common Venue,88th Most Common Venue,89th Most Common Venue,90th Most Common Venue,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Italian Restaurant,Café,Japanese Restaurant,Hotel,Breakfast Spot,Seafood Restaurant,Bakery,Beer Bar,Gastropub,Gym,Cocktail Bar,Cheese Shop,Park,Farmers Market,Art Gallery,Creperie,Hostel,Grocery Store,Yoga Studio,Jazz Club,Lounge,Molecular Gastronomy Restaurant,Movie Theater,Museum,New American Restaurant,Gourmet Shop,Eastern European Restaurant,French Restaurant,Comfort Food Restaurant,Antique Shop,BBQ Joint,Bagel Shop,Bistro,Bookstore,Church,Concert Hall,Fountain,Cosmetics Shop,Deli / Bodega,Department Store,Diner,Fish Market,Food Truck,Optical Shop,American Restaurant,Sandwich Place,Sporting Goods Shop,Thai Restaurant,Poke Place,Speakeasy,Pub,Tailor Shop,Vegetarian / Vegan Restaurant,Comedy Club,Convenience Store,Taco Place,Comic Shop,Plaza,Colombian Restaurant,Coworking Space,College Rec Center,College Gym,College Arts Building,Tanning Salon,Tea Room,Sushi Restaurant,Street Art,Supermarket,Dance Studio,Strip Club,Dessert Shop,Dim Sum Restaurant,Steakhouse,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Sri Lankan Restaurant,Electronics Store,Escape Room,Clothing Store,Theater,Chinese Restaurant,Udon Restaurant,Wine Shop,Aquarium,Arepa Restaurant,Wine Bar,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Vietnamese Restaurant,Video Game Store,Bank,Bar,Baseball Stadium
1,"CN Tower, King and Spadina, Railway Lands, Har...",Hotel,Coffee Shop,Baseball Stadium,Pizza Place,Ice Cream Shop,Restaurant,Italian Restaurant,Concert Hall,Scenic Lookout,Gym,Aquarium,Café,Sandwich Place,Brewery,Salad Place,Roof Deck,Greek Restaurant,Burger Joint,Indian Restaurant,IT Services,Park,Comedy Club,Breakfast 

In [113]:
# Run k-means to cluster the neighbourhood into  clusters.¶

# set number of clusters based on hit and trial method
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([0, 0, 0, 0, 0, 3, 0, 1, 0, 4, 0, 2], dtype=int32)

In [87]:
labels = kmeans.predict(toronto_grouped_clustering)
centroids = kmeans.cluster_centers_
labels

array([0, 0, 0, 0, 0, 3, 0, 1, 0, 4, 0, 2], dtype=int32)

In [120]:
Toronto_Neighbourhoods

,Unnamed: 0,Neighbourhood,Nei_lat,Nei_long
0,0,"Richmond, Adelaide, King",43.649972,-79.383223
1,1,"Garden District, Ryerson",43.657658,-79.378802
2,2,"CN Tower, King and Spadina, Railway Lands, Har...",43.642566,-79.387057
3,3,Berczy Park,43.648100,-79.375200
4,4,"Kensington Market, Chinatown, Grange Park",43.650883,-79.397226
5,5,"Harbourfront East, Union Station, Toronto Islands",43.640412,-79.375724
6,6,"University of Toronto, Harbord",43.662892,-79.395656
7,7,Central Bay Street,43.657298,-79.384364
8,8,"Regent Park, Harbourfront",43.660323,-79.362044
9,9,Church and Wellesley,43.664542,-79.381752


In [121]:
#removed central bay street, it appeared twice.
Toronto_Neighbourhoods= Toronto_Neighbourhoods.drop([12])


In [122]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

toronto_merged = Toronto_Neighbourhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged # check the last columns!

,Unnamed: 0,Neighbourhood,Nei_lat,Nei_long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue,51th Most Common Venue,52th Most Common Venue,53th Most Common Venue,54th Most Common Venue,55th Most Common Venue,56th Most Common Venue,57th Most Common Venue,58th Most Common Venue,59th Most Common Venue,60th Most Common Venue,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue,71th Most Common Venue,72th Most Common Venue,73th Most Common Venue,74th Most Common Venue,75th Most Common Venue,76th Most Common Venue,77th Most Common Venue,78th Most Common Venue,79th Most Common Venue,80th Most Common Venue,81th Most Common Venue,82th Most Common Venue,83th Most Common Venue,84th Most Common Venue,85th Most Common Venue,86th Most Common Venue,87th Most Common Venue,88th Most Common Venue,89th Most Common Venue,90th Most Common Venue,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,0,"Richmond, Adelaide, King",43.649972,-79.383223,0,Coffee Shop,Café,Gym,American Restaurant,Japanese Restaurant,Steakhouse,Hotel,Restaurant,Asian Restaurant,Breakfast Spot,Sushi Restaurant,Office,Gastropub,Cosmetics Shop,Seafood Restaurant,Concert Hall,Pizza Place,Bookstore,Sandwich Place,Salad Place,Pub,Plaza,Latin American Restaurant,Ramen Restaurant,Building,Burger Joint,Jazz Club,Department Store,Burrito Place,Women's Store,Mediterranean Restaurant,Food Court,Noodle House,New American Restaurant,Juice Bar,Monument / Landmark,Colombian Restaurant,Opera House,Brazilian Restaurant,Deli / Bodega,Bar,Wine Bar,Furniture / Home Store,Vegetarian / Vegan Restaurant,Art Gallery,Garden,Theater,Thai Restaurant,Tea Room,General Travel,Electronics Store,Bakery,Taco Place,Lounge,Salon / Barbershop,Gluten-free Restaurant,Beer Bar,Gym / Fitness Center,Speakeasy,Greek Restaurant,Smoke Shop,Fast Food Restaurant,Fish Market,Filipino Restaurant,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Escape Room,Dog Run,Diner,Discount Store,Farmers Market,Ethiopian Restaurant,Falafel Restaurant,Distribution Center,Dim Sum Restaurant,Cocktail Bar,Dance Studio,Bagel Shop,Board Shop,Bistro,Beer Store,Basketball Stadium,Baseball Stadium,Bank,BBQ Joint,Creperie,Auto Dealership,Arts & Crafts Store,Art Museum,Arepa Restaurant,Aquarium,Antique Shop,Boat or Ferry,Boutique,Brewery,Bubble Tea Shop,Caribbean Restaurant,Cheese Shop
1,1,"Garden District, Ryerson",43.657658,-79.378802,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Sandwich Place,Middle Eastern Restaurant,Italian Restaurant,Hotel,Theater,Ramen Resta

In [123]:
toronto_merged.to_excel('toronto_merged.xlsx')